In [1]:
# Data Collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

# Load the Dataset
data = gutenberg.raw('shakespeare-hamlet.txt')

# save to a file
with open('hamlet.txt', 'w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\navne\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


In [ ]:
# Data preprocessing

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Load the dataset
with open('hamlet.txt', 'r') as file:
    text = file.read().lower()

# Tokenize the text and creating indexes for words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words



4818

In [6]:
# Create input sequence
inputsequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        inputsequences.append(n_gram_sequence)

In [7]:
# pad sequences
max_sequence_len = max([len(x) for x in inputsequences])
max_sequence_len


14

In [9]:
input_sequences = np.array(pad_sequences(inputsequences, maxlen=max_sequence_len, padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [10]:
# create predictors and labels
import tensorflow as tf
x, y = input_sequences[:, :-1], input_sequences[:, -1]
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]])

In [11]:
y

array([ 687,    4,   45, ..., 1047,    4,  193])

In [12]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [13]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [19]:
# Train our LSTM RNN 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 13, 100)           481800    
                                                                 
 lstm_3 (LSTM)               (None, 13, 150)           150600    
                                                                 
 dropout_1 (Dropout)         (None, 13, 150)           0         
                                                                 
 lstm_4 (LSTM)               (None, 100)               100400    
                                                                 
 dense_1 (Dense)             (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Train the model
history = model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), verbose=1)

Epoch 1/50


644/644 [==============================] - 28s 35ms/step - loss: 6.8927 - accuracy: 0.0335 - val_loss: 6.7641 - val_accuracy: 0.0342
Epoch 2/50
644/644 [==============================] - 30s 46ms/step - loss: 6.4521 - accuracy: 0.0390 - val_loss: 6.8487 - val_accuracy: 0.0404
Epoch 3/50
644/644 [==============================] - 33s 51ms/step - loss: 6.3032 - accuracy: 0.0471 - val_loss: 6.9039 - val_accuracy: 0.0497
Epoch 4/50
644/644 [==============================] - 34s 53ms/step - loss: 6.1580 - accuracy: 0.0519 - val_loss: 6.9412 - val_accuracy: 0.0490
Epoch 5/50
644/644 [==============================] - 33s 51ms/step - loss: 6.0184 - accuracy: 0.0566 - val_loss: 6.9921 - val_accuracy: 0.0536
Epoch 6/50
644/644 [==============================] - 33s 51ms/step - loss: 5.8812 - accuracy: 0.0627 - val_loss: 7.0530 - val_accuracy: 0.0542
Epoch 7/50
644/644 [==============================] - 32s 49ms/step - loss: 5.7463 - accuracy: 0.0696 - val_loss: 7.0886 - val_accurac